In [1]:
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import pandas as pd
from dotenvy import load_env, read_file
import spotipy

%matplotlib inline
os.chdir('..')

# Login to Spotify app
* Use spotipy package to access Spotify developer app using credentials from .env file
* Copy URL from redirect and paste into dialog box

In [2]:
sp = spotipy.Spotify()
load_env(read_file('.env'))
cid = os.environ.get('SPOTIPY_CLIENT_ID')
secret = os.environ.get('SPOTIPY_CLIENT_SECRET')
redirect = os.environ.get('SPOTIPY_REDIRECT_URI')
username = os.environ.get('SPOTIPY_USERNAME')

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = client_credentials_manager.get_access_token()

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

# Get raw data

In [3]:
p_playlist = sp.user_playlist("federicociner", "7ieLWhjh16EPCRV721NyPl")
n_playlist = sp.user_playlist("federicociner", "5NQnP4gJMAAhmgnvYdRyqz")
features = []

p_tracks = p_playlist["tracks"]
p_songs = p_tracks["items"]
while p_tracks["next"]:
    p_tracks = sp.next(p_tracks)
    [p_songs.append(item) for item in p_tracks["items"]]

n_tracks = n_playlist["tracks"]
n_songs = n_tracks["items"]
while n_tracks["next"]:
    n_tracks = sp.next(n_tracks)
    [n_songs.append(item) for item in n_tracks["items"]]
    
p_ids = [p_songs[i]["track"]["id"] for i in range(0, len(p_songs))]
n_ids = [n_songs[i]["track"]["id"] for i in range(0, len(n_songs))]

for i in range(0, len(p_ids), 100):
    audio_features = sp.audio_features(p_ids[i:i + 100])
    for track in audio_features:
        features.append(track)
        features[-1]['class'] = 1

for i in range(0, len(n_ids), 100):
    audio_features = sp.audio_features(n_ids[i:i + 100])
    for track in audio_features:
        features.append(track)
        features[-1]['class'] = 0

feats_df = pd.DataFrame(features)
non_features = ['analysis_url', 'id', 'track_href', 'type', 'uri']
feats_df.drop(labels=non_features, axis=1, inplace=True)